<a href="https://colab.research.google.com/github/yeop-sang/Scoring_Sentence/blob/main/ScoringSentence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install konlpy
!pip install openpyxl
# 기본적인 package 설정
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings
warnings.filterwarnings(action='ignore')

from google.colab import data_table
data_table.enable_dataframe_formatter()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 4.6 MB/s eta 0:00:00


In [ ]:
from konlpy.tag import Komoran
from gensim.models import FastText


def preprocess_answer(text):
    # 1. 텍스트 정제
    preprocessed = text.strip()
    preprocessed = re.sub(r'[^\w\s]', '', preprocessed)

    # 2. 형태소 분석
    komoran = Komoran()
    tokens = komoran.morphs(preprocessed)

    # 3. 불용어 제거
    stopwords = ['은', '는', '이', '가', '을', '를']
    tokens = [token for token in tokens if token not in stopwords]

    return tokens

def visualize_preprocessing(text):
    # 전처리 단계별 토큰 수 계산
    raw_tokens = len(text.split())
    cleaned_tokens = len(re.sub(r'[^\w\s]', '', text).split())
    morphs = len(Komoran().morphs(text))
    final_tokens = len(preprocess_answer(text))

    # 시각화
    steps = ['Raw Text', 'Cleaned Text', 'Morphs', 'Final Tokens']
    counts = [raw_tokens, cleaned_tokens, morphs, final_tokens]

    plt.figure(figsize=(10, 6))
    plt.plot(steps, counts, marker='o')
    plt.title('서술형 답안 전처리 과정')
    plt.ylabel('토큰 수')
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
import pandas as pd
from google.colab import drive

# Google Drive 마운트
drive.mount('/content/drive', force_remount=True)

# Excel 파일 경로
file_path = '/content/drive/MyDrive/DataScience/examples.xlsx'

# 시트 목록 확인
excel_file = pd.ExcelFile(file_path)
print("사용 가능한 시트 목록:")
print(excel_file.sheet_names)

# 각 시트별로 DataFrame 생성
dataframes = {}

for sheet in excel_file.sheet_names:
    # 데이터 읽기
    df = pd.read_excel(
        file_path,
        sheet_name=sheet,
        header=0,
        usecols=[5,8]
    )

    # 변수명으로 저장 (sheet 이름으로 DataFrame 생성)
    dataframes[sheet] = df

    # 각 DataFrame 정보 출력
    print(f"\n=== {sheet} DataFrame ===")
    print(f"데이터 크기: {df.shape}")
    print(f"열 이름: {df.columns.tolist()}")
    print("\n미리보기:")
    print(df.head())
    print("\n결측치 현황:")
    print(df.isnull().sum())
    print("-" * 50)

Mounted at /content/drive
사용 가능한 시트 목록:
['Sheet1', 'Sheet2', 'Sheet3', 'Sheet4', 'Sheet5', 'Sheet6', 'Sheet7', 'Sheet8', 'Sheet9', 'Sheet10', 'Sheet11', 'Sheet12', 'Sheet13', 'Sheet14', 'Sheet15', 'Sheet16', 'Sheet17', 'Sheet18', 'Sheet19', 'Sheet20', 'Sheet21', 'Sheet22', 'Sheet23', 'Sheet24', 'Sheet25', 'Sheet26', 'Sheet27', 'Sheet28', 'Sheet29', 'Sheet30', 'Sheet31', 'Sheet32', 'Sheet33', 'Sheet34', 'Sheet35', 'Sheet36', 'Sheet37', 'Sheet38', 'Sheet39', 'Sheet40', 'Sheet41', 'Sheet42', 'Sheet43', 'Sheet44', 'Sheet45', 'Sheet46', 'Sheet47', 'Sheet48', 'Sheet49', 'Sheet50']

=== Sheet1 DataFrame ===
데이터 크기: (213, 2)
열 이름: ['원소 스펙트럼은 선 스펙트럼으로 나타나며 원소마다 선의 위치, 개수, 굵기가 다릅니다. 원소마다 다른 선 스펙트럼이 나타나는 이유는 무엇인가요?', '하현점수']

미리보기:
  원소 스펙트럼은 선 스펙트럼으로 나타나며 원소마다 선의 위치, 개수, 굵기가 다릅니다. 원소마다 다른 선 스펙트럼이 나타나는 이유는 무엇인가요?  \
0                                 원소마다 내는 빛이 다르기 때문에                                 
1    전자들이 높은 에너지상태에서 낮은 에너지상태로 떨어지면서 특정파장의 빛을 방출하기때문                                 
2          

In [ ]:
dataframes['Sheet1']

,"원소 스펙트럼은 선 스펙트럼으로 나타나며 원소마다 선의 위치, 개수, 굵기가 다릅니다. 원소마다 다른 선 스펙트럼이 나타나는 이유는 무엇인가요?",하현점수
0,원소마다 내는 빛이 다르기 때문에,1
1,전자들이 높은 에너지상태에서 낮은 에너지상태로 떨어지면서 특정파장의 빛을 방출하기때문,2
2,원소마다 가지고 있는 물질이 다르기 때문이다.,0
3,원소의 종류와 온도가 다 다르기 때문이다,0
4,원소의 종류나 온도가 각각 다르기 때문이다,0
...,...,...
208,스펙트럼은 원소의 종류나 온도에 따라 다르게 나타나기때문.,0
209,별빛이 지구로 오는 동안 우주공간에 있는 원소에 흡수 되어 나타나기때문이다. 또한 ...,0
210,원소마다 흡수하거나 방출하는 에너지의 양이 다르기 때문이다.,1
211,원소의 종류가 달라서,0


예외 발생: java.lang.NullPointerException
예외 발생: java.lang.NullPointerException
예외 발생: java.lang.NullPointerException
예외 발생: java.lang.NullPointerException
예외 발생: java.lang.NullPointerException
예외 발생: java.lang.NullPointerException
예외 발생: java.lang.NullPointerException
예외 발생: java.lang.NullPointerException
예외 발생: java.lang.NullPointerException
예외 발생: 'utf-8' codec can't decode byte 0xed in position 0: invalid continuation byte

=== 모즐리는 원소를 원자 번호 순서대로 나열하고 성질이 비슷한 원소가 같은 세로줄에 오도록 배치하여 오늘날의 주기율표를 만들었습니다. 주기율표 상에서 족은 무엇을 말하나요? 열의 상위 20개 빈도 명사 ===
세로줄: 90
성질: 48
원소: 38
족: 35
주기율표: 34
것: 26
수: 22
줄: 17
원자가 전자: 16
세로: 11
말: 11
전자: 11
가로: 9
원자: 9
개수: 8
속: 7
개: 7
외각: 7
화학적 성질: 7
최외각전자: 7
예외 발생: java.lang.NullPointerException
예외 발생: java.lang.NullPointerException
예외 발생: java.lang.NullPointerException
예외 발생: 'utf-8' codec can't decode byte 0xed in position 0: invalid continuation byte
예외 발생: java.lang.NullPointerException
예외 발생: java.lang.NullPointerException
예외 발생: java.lang.NullPointerException
예

In [ ]:
# prompt: dataframes를 모두 순회하면서, 전처리 과정으로 1번째 column에 있는 자연어 값을 token화하려고 해. token화 한 값을 dataframes의 해당 dataframe의 새로운 column을 추가해서 넣어줘.

# 기존 코드에 이어서 작성합니다.

for sheet_name, df in dataframes.items():
    # 1번째 column 이름을 가져옵니다. (가정: 첫 번째 열의 이름이 'text'입니다.)
    first_column_name = df.columns[0]

    # 새로운 column을 추가합니다.
    df['tokens'] = df[first_column_name].apply(preprocess_answer)

    # 결과 출력 (선택 사항)
    print(f"\n=== {sheet_name} DataFrame (tokenized) ===")
    print(df.head())

In [ ]:
dataframes['Sheet1']